In [3]:
import numpy as np
import warnings
import dill
import pandas as pd
# TODO Need to import spatial tSIR module. is there a better way to do this??
import sys
from my_scripts.spatial_tsir import *
from my_scripts.optimization import vacc
import datetime
import os
import multiprocess
from multiprocess import Pool

Test batched evaluation

In [2]:
%cd ..

/home/prasanna/Projects/CS_4501_Fall22_RL_Project


In [4]:
vacc_df = {
        'pop': [1200, 1200, 4000, 2000, 5000],
        'vacc': [0.9,0.9,0.9,0.9, 0.75]
}
vacc_df = pd.DataFrame(vacc_df)

# set up distance matrix
# all pairwise distances
dist_mat = [
        [0,1,5,5,1],
        [1,0,5,5,1],
        [5,5,0,10,1],
        [5,5,10,0,1],
        [5,5,10,1,0]
]

# setup the configuration
# for the disease simulation
tsir_config = {
    "iters": 75,    # number of iterations to run sim for
    "tau1": 0.7,    # gravity model parameters
    "tau2": 1.2,
    "rho": 0.97,
    "theta": 0.05,
    "alpha": 0.97, # mixing rate
    "beta": 7      # disease infectiousness
}
# arguments for optimizer oracle
sim_params = {
        'config':tsir_config,  # contains all disease parameters
        'pop':vacc_df,
        'distances':np.array(dist_mat)
}

# optimizer oracle configuration
opt_config = {
    'obj':"attacksize",   # objective function
    'V_repr':"ratio",     # represent vacc rates as a ratio: [0,1]
    'constraint_bnd':0.05, # set c=0.05 (percentage can go down by 5%)
    'constraint_type':'ineq'
}

I = np.array([0,0,1,0,0])   # seeding: set outbreak to begin in district 4

initial_state = np.zeros((len(vacc_df.index),2))
initial_state[:, 0] = I
#initial_state[:,0] = seed_prime
initial_state[:, 1] = np.round(np.array(vacc_df['pop'])*np.array(vacc_df['vacc']))

In [5]:
pool = spatial_tSIR_pool(config=tsir_config,patch_pop=vacc_df, initial_state=initial_state,n_sim=100,distances=np.array(dist_mat))

In [6]:
p = multiprocess.Pool(10)
result=pool.run_simulation_async(pool=p)

In [7]:
p.close()

In [8]:
vacc_df = {
        'pop': [1200, 1200, 4000, 2000, 5000],
        'vacc': [0.9,0.9,0.9,0.9, 0.75]
}
vacc_df = pd.DataFrame(vacc_df)

# set up distance matrix
# all pairwise distances
dist_mat = [
        [0,1,5,5,1],
        [1,0,5,5,1],
        [5,5,0,10,1],
        [5,5,10,0,1],
        [5,5,10,1,0]
]

# setup the configuration
# for the disease simulation
tsir_config = {
    "iters": 75,    # number of iterations to run sim for
    "tau1": 0.7,    # gravity model parameters
    "tau2": 1.2,
    "rho": 0.97,
    "theta": 0.05,
    "alpha": 0.97, # mixing rate
    "beta": 7      # disease infectiousness
}
# arguments for optimizer oracle
sim_params = {
        'config':tsir_config,  # contains all disease parameters
        'pop':vacc_df,
        'distances':np.array(dist_mat)
}

# optimizer oracle configuration
opt_config = {
    'obj':"attacksize",   # objective function
    'V_repr':"ratio",     # represent vacc rates as a ratio: [0,1]
    'constraint_bnd':0.05, # set c=0.05 (percentage can go down by 5%)
    'constraint_type':'ineq'
}

I = np.array([0,0,1,0,0])   # seeding: set outbreak to begin in district 4

v = vacc.VaccProblemLAMCTSWrapper(
        opt_config = opt_config, 
        V_0=vacc_df['vacc'], 
        seed = I,
        sim_config = tsir_config, 
        pop = vacc_df, 
        distances = np.array(dist_mat),
        cores=5, n_sim=10,
        output_dir = "/tmp/",
        negate=True,
        scale=1,
    name="d"
    )

In [17]:
v(np.array([0.05,0.05,0.05,0.05,0.05]))

-0.04477611940298507

In [34]:
p_list = [Pool(20) for i in range(20)]
xs = [np.ones(5)*x for x in np.linspace(0,0.05,20)]
result=v(xs,pool=p_list,batch=True)

In [35]:
result

[-0.026873134328358212,
 -0.027611940298507463,
 -0.031417910447761196,
 -0.032238805970149255,
 -0.032985074626865674,
 -0.03380597014925373,
 -0.03455223880597015,
 -0.03537313432835821,
 -0.03611940298507463,
 -0.036940298507462686,
 -0.037686567164179105,
 -0.03466417910447761,
 -0.03533582089552239,
 -0.03607462686567164,
 -0.036746268656716416,
 -0.03748507462686567,
 -0.03815671641791045,
 -0.038895522388059704,
 -0.03956716417910448,
 -0.04030597014925373]

In [9]:
xs = [np.ones(5)*0.05]*5

In [10]:
xs

[array([0.05, 0.05, 0.05, 0.05, 0.05]),
 array([0.05, 0.05, 0.05, 0.05, 0.05]),
 array([0.05, 0.05, 0.05, 0.05, 0.05]),
 array([0.05, 0.05, 0.05, 0.05, 0.05]),
 array([0.05, 0.05, 0.05, 0.05, 0.05])]

In [33]:
np.linspace(0,1,20)

array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])